In [7]:
import xml.etree.ElementTree as ET
import pandas as pd

# Define the namespace
namespace = {'ns': 'urn:cwa:gov:tw:cwacommon:0.1'}

# time scale
n_years = 9
years = [2014+x for x in range(n_years)]
months = [f"{i:02d}" for i in range(1, 13)]


# concatenate all dfs
all_dfs = pd.DataFrame(columns=[
    'StationID',
    'StationName',
    'StationNameEN',
    'StationAttribute',
    'AirTemperatureMean',
    'AirTemperatureMax',
    'AirTemperatureMaxDate',
])


for year in years:
    print(f'\n===================================== {year} =====================================\n')

    for month in months:        
        
        # Parse the XML file
        tree = ET.parse(f'./data/{year}/mn_Report_{year}{month}.xml')
        root = tree.getroot()

        # Create lists to store data
        data = []

        # Iterate through each 'location' element
        for location in root.findall('.//ns:location', namespace):
            station_id = location.find('.//ns:StationID', namespace).text
            station_name = location.find('.//ns:StationName', namespace).text
            station_name_en = location.find('.//ns:StationNameEN', namespace).text
            station_attribute = location.find('.//ns:StationAttribute', namespace).text

            air_temperature_mean = location.find('.//ns:Mean', namespace).text
            air_temperature_max = location.find('.//ns:Maximum', namespace).text
            air_temperature_max_date = location.find('.//ns:MaximumDate', namespace).text

            # Append data to the list
            data.append({
                'StationID': station_id,
                'StationName': station_name,
                'StationNameEN': station_name_en,
                'StationAttribute': station_attribute,
                'AirTemperatureMean': air_temperature_mean,
                'AirTemperatureMax': air_temperature_max,
                'AirTemperatureMaxDate': air_temperature_max_date
            })

        # Create a DataFrame
        df = pd.DataFrame(data)
        
        all_dfs = pd.concat([all_dfs, df], ignore_index=True)

all_dfs


===================================== 2014 =====================================


===================================== 2015 =====================================


===================================== 2016 =====================================


===================================== 2017 =====================================


===================================== 2018 =====================================


===================================== 2019 =====================================


===================================== 2020 =====================================


===================================== 2021 =====================================


===================================== 2022 =====================================



,StationID,StationName,StationNameEN,StationAttribute,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
0,466880,板橋,BANQIAO,署屬有人氣象站,16.2,26.6,2014-01-30
1,466900,淡水,TAMSUI,署屬有人氣象站,16.2,27.0,2014-01-31
2,466910,鞍部,ANBU,署屬有人氣象站,10.2,20.6,2014-01-02
3,466920,臺北,TAIPEI,署屬有人氣象站,16.8,26.8,2014-01-30
4,466930,竹子湖,ZHUZIHU,署屬有人氣象站,12.0,22.4,2014-01-30
...,...,...,...,...,...,...,...
3053,467610,成功,CHENGGONG,署屬有人氣象站,20.3,29.7,2022-12-03
3054,467620,蘭嶼,LANYU,署屬有人氣象站,19.2,24.8,2022-12-04
3055,467650,日月潭,SUN MOON LAKE,署屬有人氣象站,15.2,26.5,2022-12-03
3056,467660,臺東,TAITUNG,署屬有人氣象站,20.9,28.5,2022-12-03


In [8]:
import os
import pandas as pd

# Assuming all_dfs is your combined DataFrame

# Group by 'StationName'
grouped = all_dfs.groupby('StationName')

# Accessing each group
for name, group in grouped:
    print(f"Station: {name}")
    
    if name == '田中':
        print("資訊不足\n")
        continue
    
    
    station_name_folder = f'./{name}'
    if not os.path.isdir(station_name_folder):
        os.mkdir(station_name_folder)


    # Convert 'AirTemperatureMaxDate' to datetime
    group['AirTemperatureMaxDate'] = pd.to_datetime(group['AirTemperatureMaxDate'])

    # Create daily-like entries for the entire month
    df_daily = group.assign(AirTemperatureMaxDate=group['AirTemperatureMaxDate'].apply(
        lambda x: pd.date_range(start=x.replace(day=1), end=x.replace(day=1) + pd.offsets.MonthEnd(0), freq='D')
    )).explode('AirTemperatureMaxDate').reset_index(drop=True)

    # Sort date from earliest to latest
    df_daily = df_daily.sort_values(by='AirTemperatureMaxDate')

    # Drop duplicate-date rows
    df_daily = df_daily.drop_duplicates(subset=['AirTemperatureMaxDate'])
    
    # Select desired columns
    df_daily = df_daily.drop(columns=['StationID', 'StationNameEN', 'StationAttribute'])

    # Group df_daily by year
    grouped_by_year = df_daily.groupby(df_daily['AirTemperatureMaxDate'].dt.year)

    # Accessing each year within the station
    for year, df_year in grouped_by_year:
        print(f"  Year: {year}")

        # Display the result for each year
        display(df_year)
        
        # save it as csv
        df_year.to_csv(f'{station_name_folder}/{year}AirTemperature.csv', index=False)


    print("\n")


Station: 嘉義
  Year: 2014


,StationName,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
0,嘉義,16.1,27.2,2014-01-01
1,嘉義,16.1,27.2,2014-01-02
2,嘉義,16.1,27.2,2014-01-03
3,嘉義,16.1,27.2,2014-01-04
4,嘉義,16.1,27.2,2014-01-05
...,...,...,...,...
360,嘉義,16.9,27.4,2014-12-27
361,嘉義,16.9,27.4,2014-12-28
362,嘉義,16.9,27.4,2014-12-29
363,嘉義,16.9,27.4,2014-12-30


  Year: 2015


,StationName,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
365,嘉義,16.8,28.6,2015-01-01
366,嘉義,16.8,28.6,2015-01-02
367,嘉義,16.8,28.6,2015-01-03
368,嘉義,16.8,28.6,2015-01-04
369,嘉義,16.8,28.6,2015-01-05
...,...,...,...,...
725,嘉義,19.8,30.2,2015-12-27
726,嘉義,19.8,30.2,2015-12-28
727,嘉義,19.8,30.2,2015-12-29
728,嘉義,19.8,30.2,2015-12-30


  Year: 2016


...

,StationName,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
2191,高雄,20.9,29.0,2020-01-01
2192,高雄,20.9,29.0,2020-01-02
2193,高雄,20.9,29.0,2020-01-03
2194,高雄,20.9,29.0,2020-01-04
2195,高雄,20.9,29.0,2020-01-05
...,...,...,...,...
2552,高雄,22.3,28.6,2020-12-27
2553,高雄,22.3,28.6,2020-12-28
2554,高雄,22.3,28.6,2020-12-29
2555,高雄,22.3,28.6,2020-12-30


  Year: 2021


,StationName,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
2557,高雄,18.3,26.3,2021-01-01
2558,高雄,18.3,26.3,2021-01-02
2559,高雄,18.3,26.3,2021-01-03
2560,高雄,18.3,26.3,2021-01-04
2561,高雄,18.3,26.3,2021-01-05
...,...,...,...,...
2917,高雄,21.2,28.1,2021-12-27
2918,高雄,21.2,28.1,2021-12-28
2919,高雄,21.2,28.1,2021-12-29
2920,高雄,21.2,28.1,2021-12-30


  Year: 2022


,StationName,AirTemperatureMean,AirTemperatureMax,AirTemperatureMaxDate
2922,高雄,20.1,27.7,2022-01-01
2954,高雄,19.1,27.0,2022-01-02
2955,高雄,19.1,27.0,2022-01-03
2956,高雄,19.1,27.0,2022-01-04
2957,高雄,19.1,27.0,2022-01-05
...,...,...,...,...
3313,高雄,19.0,30.0,2022-12-27
3314,高雄,19.0,30.0,2022-12-28
3315,高雄,19.0,30.0,2022-12-29
3316,高雄,19.0,30.0,2022-12-30


In [9]:
# directory = ''
# folders = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory, d))]